# Лабораторная работа №2

# Методы многомерной оптимизации

In [21]:
from typing import Callable
from typing import List

import numpy as np

from scipy.optimize import approx_fprime
from scipy.optimize import minimize_scalar

## Инициализация оптимизируемой функции

In [22]:
def f(x: List[float]) -> float:
    A = 10
    a = 3
    b = 1
    
    return A - ((x[0] - a) * np.exp(a - x[0])) - ((x[1] - b) * np.exp(b - x[0]))

## Метод Гаусса-Зейделя

#### Решение системы линейных уравнений методом Гаусса-Зейделя

In [23]:
def gauss_seidel(A: np.ndarray, b: np.ndarray, x0: np.ndarray, tol=1e-10, max_iter=1000):
    """
    Решение системы линейных уравнений Ax=b методом Гаусса-Зейделя.

    Parameters:
        A: np.array, матрица системы
        b: np.array, столбец свободных членов
        x0: np.array, начальное приближение
        tol: float, точность вычисления решения
        max_iter: int, максимальное число итераций

    Returns:
        x: np.array, решение системы
    """

    n = A.shape[0]
    x = x0.copy()

    for k in range(max_iter):
        for i in range(n):
            x[i] = (b[i] - np.dot(A[i, :i], x[:i]) - np.dot(A[i, i+1:], x[i+1:])) / A[i, i]
        
        if np.linalg.norm(A @ x - b) < tol:
            break
    
    return x

In [24]:
def gauss_seidel_optimization(f: Callable, x0: np.ndarray, tol=1e-10, max_iter=1000):
    """
    Оптимизация многомерной функции методом Гаусса-Зейделя.

    Parameters:
        f: function, оптимизируемая функция
        x0: np.array, начальное приближение
        tol: float, точность вычисления решения
        max_iter: int, максимальное число итераций

    Returns:
        x: np.array, оптимальное решение
    """

    n = len(x0)
    x = x0.copy()

    for k in range(max_iter):
        # вычисляем градиент и гессиан
        grad = approx_fprime(x, f, 1e-8)
        hess = approx_fprime(x, lambda x: approx_fprime(x, f, 1e-8), 1e-8)

        # строим матрицу и правую часть уравнения
        A = np.eye(n) - hess
        b = x - grad

        # решаем систему уравнений методом Гаусса-Зейделя
        x_new = gauss_seidel(A, b, x)

        if np.linalg.norm(x_new - x) < tol:
            break

        x = x_new

    return x

In [25]:
x0 = np.array([-2, -3])
x_opt = gauss_seidel_optimization(f, x0)

print(x_opt)

[0 0]


## Метод сопряжённых направлений Пауэлла

In [26]:
def powell_optimization(f: Callable, x0: np.ndarray, tol=1e-10, max_iter=1000):
    """
    Оптимизация многомерной функции методом сопряженных направлений (алгоритм Пауэлла).

    Parameters:
        f: function, оптимизируемая функция
        x0: np.array, начальное приближение
        tol: float, точность вычисления решения
        max_iter: int, максимальное число итераций

    Returns:
        x: np.array, оптимальное решение
    """

    n = len(x0)
    x = x0.copy()
    delta = np.zeros(n)
    E = np.eye(n)

    for k in range(max_iter):
        # вычисляем градиент в текущей точке
        grad = approx_fprime(x, f, 1e-8)

        # сопряженные направления
        d = E.copy()
        for i in range(n):
            d[i] = delta[i] * E[i]

        # оптимизируем квадратичную функцию вдоль сопряженных направлений
        alpha = np.zeros(n)
        for i in range(n):
            func = lambda a: f(x + a*d[i])
            alpha[i] = minimize_scalar(func).x

        # переход к новой точке
        x_new = x + np.dot(alpha, d)

        if np.linalg.norm(x_new - x) < tol:
            break

        # вычисляем новое направление
        delta = x_new - x
        x = x_new

    return x


In [35]:
x0 = np.array([0, 0])
x_opt = powell_optimization(f, x0)

print(x_opt)

[0 0]


## Метод наискорейшего спуска (Коши)

In [28]:
def steepest_descent_optimization(f: Callable, x0: np.ndarray, tol=1e-10, max_iter=1000):
    """
    Оптимизация многомерной функции методом наискорейшего спуска (метод Коши).

    Parameters:
        f: function, оптимизируемая функция
        x0: np.array, начальное приближение
        tol: float, точность вычисления решения
        max_iter: int, максимальное число итераций

    Returns:
        x: np.array, оптимальное решение
    """

    n = len(x0)
    x = x0.copy()

    for k in range(max_iter):
        # вычисляем градиент в текущей точке
        grad = approx_fprime(x, f, 1e-8)

        # направление движения
        d = -grad

        # оптимизируем функцию в направлении движения
        func = lambda a: f(x + a*d)
        alpha = minimize_scalar(func).x

        # переход к новой точке
        x_new = x + alpha*d

        if np.linalg.norm(x_new - x) < tol:
            break

        x = x_new

    return x

In [34]:
x0 = np.array([0, 0])
x_opt = steepest_descent_optimization(f, x0)

print(x_opt)

[217.45502464   7.11655494]
